# <font color='blue'> DSA - Python Fundamentos - Capítulo 6 </font>

## Stream de Dados do Twitter com MongoDB, Pandas e Scikit Learn

In [ ]:
# Importando Tweepy, Datetime e Json
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime
import json

In [ ]:
consumer_key = ''

In [ ]:
consumer_secret = ''

In [ ]:
access_token = ''

In [ ]:
access_token_secret = ''

In [ ]:
# Criando as chaves de autenticação
auth = OAuthHandler(consumer_key, consumer_secret)

In [ ]:
auth.set_access_token(access_token, access_token_secret)

In [ ]:
# Criando uma classe para capturar os stream de dados do Twitter e armazenar no MongoDB
class MyListener(StreamListener):
    def on_data(self, dados):
        tweet = json.loads(dados)
        created_at = tweet["created_at"]
        id_str = tweet["id_str"]
        text = tweet["text"]
        obj = {"created_at": created_at, "id_str": id_str, "text": text}
        tweetind = col.insert_one(obj).inserted_id
        print(obj)
        return True

In [ ]:
mylistener = MyListener()

In [ ]:
mystream = Stream(auth, listener= mylistener)

In [ ]:
from pymongo import MongoClient

In [ ]:
client = MongoClient('localhost', 27017)

In [ ]:
db = client.twitterdb

In [ ]:
# Criando collection tweets
col = db.tweets

In [ ]:
# Criando palavras chaves para buscar nos tweets
keywords = ['Big Data', 'Python', 'Data Mining', 'Data Science']

In [ ]:
# Iniciando o filtro e gravando os tweets no MongoDB
mystream.filter(track=keywords)